# Исследование надежности заемщиков.

<b>Цель данного проекта выяснить:</b>
   * влияет ли семейное положение на факт погошения кредита в срок 
   * влияет ли количество детей на факт погошения кредита в срок 

<b>Пояснение к данным:</b>
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

<b>План работы:</b>
- Ознакомление с данными
- Предобработка данных
- Анализ данных

## Шаг 1. Обзор данных

In [1181]:
import pandas as pd
from pymystem3 import Mystem

In [1182]:
df = pd.read_csv('/datasets/data.csv')

#Знакомство с таблицей
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество пропусков в таблице

In [1183]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [1184]:
df_null_days = df['days_employed'].isna().sum()
df_null_income = df['total_income'].isna().sum()

**Причины пропусков**

Как правило причинами является человеческий фактор или техническая ошибка, когда вводили результаты в таблицу. Бывают и другие причины, но для ответа на этот вопрос нужно знать каждый конкретный случай.

Данные для расчета доли пропущенных значений:

In [1185]:
count_days_employed = df['days_employed'].count()
count_total_income = df['total_income'].count()

Доля пропущенных значений:

In [1186]:
print('Доля пропущенных значений в days_employed: {:.0%}'.format(df_null_days/count_days_employed))
print('Доля пропущенных значений в total_income: {:.0%}'.format(df_null_income/count_total_income))

Доля пропущенных значений в days_employed: 11%
Доля пропущенных значений в total_income: 11%


## Шаг 2.1 Заполнение пропусков медианным значением

Заполнение пропусков медианным значением:

In [1187]:
df = df.fillna({'total_income': df.groupby('income_type')['total_income'].transform('median')})
df = df.fillna({'days_employed': df.groupby('income_type')['days_employed'].transform('median')})

Проверка корректности применения метода:

In [1188]:
print(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Использование медианы**

Использование медианы лучще для заполнения, потому что в таком случае будут менишие искажения при расчетах, если у нас имеются "выбросы"

## Шаг 2.2 Проверка данных на аномалии и исправления.

### Нахождение аномальных значений

Обзор таблици для поиска аномальных значений:

In [1189]:
df_columns = df.columns

for i in df_columns:
    print(df[i].value_counts().head(10))
    print(' ')
    print('Максимальное значение: ',df[i].max())
    print('-----------------------------------')

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
 
Максимальное значение:  20
-----------------------------------
-1574.202821      1105
-1547.382223       509
 365213.306266     414
-2689.368353       147
-520.848083          2
 396078.542064       1
-4906.125062         1
-4236.274243         1
-6620.396473         1
-1238.560080         1
Name: days_employed, dtype: int64
 
Максимальное значение:  401755.40047533
-----------------------------------
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
Name: dob_years, dtype: int64
 
Максимальное значение:  75
-----------------------------------
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОК

### Обработка аномальных значений в `gender`

Удаление одной строки с неопределяемым гендером:

In [1190]:
df = df[df['gender']!='xna']

Проверка корректности удаления:

In [1191]:
print(df.shape[0])
print(df['gender'].value_counts())

21525
F      14236
M       7288
XNA        1
Name: gender, dtype: int64


### Обработка аномальных значений в `children`

Замена ошибочных значений:

In [1192]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1
print(df['children'].value_counts().head(10))

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


### Обработка аномальных значений в `days_employed`

**Найденные проблемы**

- отрицательный стаж
- нецелое число стажа в днях
- слишком большой стаж у некоторых значений (порядка 1000 лет)

Для нашего анализа данный столбец не нужен, поэтому просто удалим его

In [1193]:
df = df.drop(columns = ['days_employed'], axis = 1)

## Шаг 2.3. Изменение типов данных в столбце `total_income`

In [1194]:
selected_dtype_float = df.select_dtypes(include='float')
mean_usage_float = selected_dtype_float.memory_usage(deep=True).mean()
mean_usage_mb_float = mean_usage_float / 1024 ** 2
print("Average memory usage: {:03.2f} MB".format(mean_usage_mb_float))

Average memory usage: 0.16 MB


In [1195]:
df['total_income'] = df['total_income'].astype('uint32')

selected_dtype_unit = df.select_dtypes(include='uint32')
mean_usage_unit = selected_dtype_unit.memory_usage(deep=True).mean()
mean_usage_mb_unit = mean_usage_unit / 1024 ** 2
print("Average memory usage: {:03.2f} MB".format(mean_usage_mb_unit))

Average memory usage: 0.12 MB


In [1196]:
print(f'Сэкономлено: {100-(mean_usage_mb_unit*100/mean_usage_mb_float)}%')

Сэкономлено: 25.0%


## Шаг 2.4. Удаление дубликатов.

Для поиска дубликатов надо привести все значения к одному виду. 
Переведем столбци с типом "object" в нижний регистр:

In [1197]:
object_type = ['education', 'family_status', 'gender', 'income_type', 'purpose']
for i in object_type:
    df[i] = df[i].str.lower()

Посмотрим на количество дубликатов:

In [1198]:
print(df.duplicated().sum())

71


Удалим явные дубликаты:

In [1199]:
df = df.drop_duplicates()

Снова проверим количество дубликатов:

In [1200]:
print(df.duplicated().sum())

0


In [1201]:
df_reserve = df #в конце объяснение зачем это. Техническая проблема. Не пойму как решить.

## Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [1202]:
df_education = df[['education', 'education_id']]
df_family = df[['family_status', 'family_status_id']]

df = df.drop(columns = ['education', 'family_status'], axis = 1)
print(df_family.head(10))

      family_status  family_status_id
0   женат / замужем                 0
1   женат / замужем                 0
2   женат / замужем                 0
3   женат / замужем                 0
4  гражданский брак                 1
5  гражданский брак                 1
6   женат / замужем                 0
7   женат / замужем                 0
8  гражданский брак                 1
9   женат / замужем                 0


## Шаг 2.6. Категоризация дохода.

Создание функции для категоризации по доходу:

In [1203]:
def category (income):
    if income <= 30000:
        return 'E'
    if income >=30001 and income <= 50000:
        return 'D'
    if income >=50001 and income <= 200000:
        return 'C'
    if income >=200001 and income <= 1000000:
        return 'B'
    if income >=1000001:
        return 'A'

Создание нового столбца с категоризацией по доходу:

In [1204]:
df['total_income_category'] = df['total_income'].apply(category)

## Шаг 2.7. Категоризация целей кредита.

Создание функции для категоризации по цели кредита:

In [1205]:
def purpose_credit(purpose):
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'образ' in purpose:
        return 'проведение свадьбы'
    elif 'свадь' in purpose:
        return 'получение образования'

Создание нового столбца с категоризацией по цели кредита:

In [1206]:
df['purpose_category'] = df['purpose'].apply(purpose_credit)

Проверка результата:

In [1207]:
print(df['purpose_category'].head(10))

0    операции с недвижимостью
1      операции с автомобилем
2    операции с недвижимостью
3          проведение свадьбы
4       получение образования
5    операции с недвижимостью
6    операции с недвижимостью
7          проведение свадьбы
8       получение образования
9    операции с недвижимостью
Name: purpose_category, dtype: object


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создание новой рабочей таблици:

In [1208]:
df_children_debt = df[['children', 'debt']]

Группировка по количеству детей. Создание новых колоное для дальнейшего расчета:

In [1209]:
children_with_debt = df_children_debt.groupby('children').agg({'debt':['count','sum']})

In [1210]:
children_with_debt['overdue_loan'] = round(children_with_debt['debt']['sum']*100/children_with_debt['debt']['count'], 2)
children_with_debt.sort_values(by = 'overdue_loan', ascending = False)

debt       overdue_loan
          count   sum             
children                          
4            41     4         9.76
2          2128   202         9.49
1          4855   445         9.17
3           330    27         8.18
0         14091  1063         7.54
5             9     0         0.00

##### Вывод 1:

Заемщики с детьми возвращают кредиты реже, чем заемщики без детей. 
По заемщикам с 5 детьми нельзя делать выводы. Выборка слишком маленькая. 
Тоже самое касается заемщиков с 4 детьми.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [1212]:
df_rel = df_reserve[['family_status', 'debt']]

In [1213]:
df_rel_group = df_rel.groupby('family_status').agg({'debt':['count','sum']})
df_rel_group['overdue_loan'] = round(df_rel_group['debt']['sum']*100/df_rel_group['debt']['count'], 2)
df_rel_group.sort_values(by = 'overdue_loan', ascending = False)

debt      overdue_loan
                       count  sum             
family_status                                 
не женат / не замужем   2810  274         9.75
гражданский брак        4151  388         9.35
женат / замужем        12339  931         7.55
в разводе               1195   85         7.11
вдовец / вдова           959   63         6.57

##### Вывод 2:

Одинокие люди реже возвращают кредиты, чем люди с партнером.

##### Вопрос 3: Есть ли зависимость между уровнем дохада и возвратом кредита в срок?

In [1214]:
df_total_income = df[['total_income_category','debt']]
df_total_income_group = df_total_income.groupby('total_income_category').agg({'debt':['count', 'sum']})
df_total_income_group['overdue_loan']= round(df_total_income_group['debt']['sum']*100/df_total_income_group['debt']['count'], 2)
df_total_income_group.sort_values(by = 'overdue_loan', ascending = False)

debt       overdue_loan
                       count   sum             
total_income_category                          
E                         22     2         9.09
C                      16015  1360         8.49
A                         25     2         8.00
B                       5042   356         7.06
D                        350    21         6.00

##### Вывод 3:

Люди с достатком от 50001 - 200000 реже остальных категорий возвращают кредит.

Из данных можно сделать заключение, что люди с низким достатком реже возвращают кредиты, чем остальные категории заемщиков. Но для данного вывода мало оснований. В категории 'Е' всего 22 заемщика. Выборка слишком маленькая.
Тоже самое касается и категории с самым высоким достатком.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [1215]:
df_purpose = df[['purpose_category', 'debt']]
df_purpose_group = df_purpose.groupby('purpose_category').agg({'debt':['count', 'sum']})
df_purpose_group['overdue_loan'] = round(df_purpose_group['debt']['sum']*100/df_purpose_group['debt']['count'], 2)
df_purpose_group.sort_values(by = 'overdue_loan', ascending = False)

debt      overdue_loan
                          count  sum             
purpose_category                                 
операции с автомобилем     4306  403         9.36
проведение свадьбы         4013  370         9.22
получение образования      2324  186         8.00
операции с недвижимостью  10811  782         7.23

##### Вывод 4:

Автокредиты и кредиты на свадьбу возвращают реже, чем остальные виды кредитов.

## Общий вывод:

Самый надежный заемщик выглядит следующи образом:
Человек без детей, женат/замужем, с достатком от 200001 - 1000000, с целью получения кредита для недвижимости.

Самый ненадежный заемщик выглядит следующи образом:
Человек с 2 детьми, без мужа/жены, с достатком 50001 - 200000, с целью получения кредита для операций с автомобилем или на свадьбу.